In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import scipy.stats
import statsmodels.api as sm
import os
import datetime as dt

In [2]:
Changwon_Tmap_df = pd.read_csv('C:\\Users\\user1\\Desktop\\창원_빅데이터_공모전\\Preprocessed_data\\Changwon_Tmap_df_ver3.csv')
del Changwon_Tmap_df['Unnamed: 0']
Changwon_Tmap_df.head()

,일자,지역(시도),지역(시군구),검색지명,검색지유형1,검색지유형2,검색지유형3,검색지랭킹(시군구내),요일,주말/주중,휴일,년도,월
0,2019-01-11,경남,창원시 성산구,창원경상대학교병원,의료편의,의료시설,종합병원,1,금요일,0,0,2019,1
1,2019-01-11,경남,창원시 성산구,반림트리비앙아파트,건물/시설,주요건물,아파트단지,2,금요일,0,0,2019,1
2,2019-01-11,경남,창원시 성산구,롯데백화점창원점,쇼핑,대형유통점,백화점,3,금요일,0,0,2019,1
3,2019-01-11,경남,창원시 성산구,반송노블파크아파트,건물/시설,주요건물,아파트단지,4,금요일,0,0,2019,1
4,2019-01-11,경남,창원시 성산구,LG전자창원2공장,기업/단체,주요그룹,LG,5,금요일,0,0,2019,1


In [3]:
# 창원에서 Tmap으로 검색된 공원의 종류 = 17 
len(Changwon_Tmap_df[Changwon_Tmap_df['검색지유형3']=='공원']['검색지명'].value_counts())

17

In [32]:
Changwon_Tmap_df['지역(시군구)'].value_counts()

창원시 마산합포구    17310
창원시 의창구      17310
창원시 진해구      17310
창원시 성산구      17310
창원시 마산회원구    17310
Name: 지역(시군구), dtype: int64

In [4]:
Changwon_Tmap_df[Changwon_Tmap_df['검색지유형3']=='공원']['검색지명'].value_counts()

북면수변생태공원       150
창원해양공원          99
장미공원            69
용지공원            52
진해해양공원          46
진해내수면환경생태공원     37
가포해안변공원         34
창원단감테마공원        30
제황산공원           23
사화공원            14
본포수변생태공원         5
창원수목원            3
만날공원             2
산호공원             2
장복산공원            2
대산플라워랜드          1
삼계근린공원           1
Name: 검색지명, dtype: int64

In [5]:
address = ['경남 창원시 의창구 북면 외산리 108',
           '경남 창원시 진해구 명동로 62',
           '경남 창원시 성산구 가음동 31',
          '경남 창원시 의창구 용지로169번길 26',
          '경남 창원시 진해구 명동로 62',
          '경남 창원시 진해구 여명로71번길 65',
          '경남 창원시 마산합포구 가포동 산 82',
          '경남 창원시 의창구 동읍 화양리 680',
          '경남 창원시 진해구 제황산동 28-5',
          '경남 창원시 의창구 중동 145',
          '경남 창원시 의창구 동읍 본포리',
          '경남 창원시 의창구 삼동동 산 14-1',
          '경남 창원시 마산합포구 산호동 247',
          '경남 창원시 마산합포구 월영동 306',
          '경남 창원시 진해구 태백동 산 52-1',
          '경남 창원시 의창구 대산면 모산리 4-10',
          '경남 창원시 마산회원구 내서읍 광려로 8']

In [6]:
import requests
from urllib.parse import urlparse
import pandas as pd
import json

In [21]:
longtitude = []
latitude = []
for i in address:
    try:
        url = 'https://dapi.kakao.com/v2/local/search/address.json?query='+i
        result = requests.get(urlparse(url).geturl(),
                            headers={'Authorization':'KakaoAK 55e6a824ef9bcae9055f83c4d5672f0d'})
        json_obj = result.json()
        longtitude.append(json_obj['documents'][0]['address']['x'])
        latitude.append(json_obj['documents'][0]['address']['y'])
    except:
        longtitude.append(0)
        latitude.append(0)
print(longtitude)
print(latitude)

['128.609790815343', '128.719347539462', '128.690275383692', '128.681022931214', '128.719347539462', '128.660005160088', '128.589396857646', 0, '128.664434312791', 0, '128.649454269739', '128.655821825524', 0, '128.548412673191', '128.668154472564', '128.716937256389', '128.503100300431']
['35.3805879621255', '35.0921817074027', '35.2074243862455', '35.2320799258082', '35.0921817074027', '35.1640346085716', '35.1583373818969', 0, '35.1476322490856', 0, '35.3658963163472', '35.2281312529098', 0, '35.1842878865456', '35.1731382435751', '35.3562964732157', '35.2260903004507']


In [22]:
park_coorinate = pd.DataFrame({'count':Changwon_Tmap_df[Changwon_Tmap_df['검색지유형3']=='공원']['검색지명'].value_counts(),
                              'address':address,
                              'longtitude':longtitude,
                              'latitude':latitude})
park_coorinate.reset_index(inplace=True)
park_coorinate.rename(columns={'index':'park'},inplace=True)
park_coorinate

,park,count,address,longtitude,latitude
0,북면수변생태공원,150,경남 창원시 의창구 북면 외산리 108,128.609790815343,35.3805879621255
1,창원해양공원,99,경남 창원시 진해구 명동로 62,128.719347539462,35.0921817074027
2,장미공원,69,경남 창원시 성산구 가음동 31,128.690275383692,35.2074243862455
3,용지공원,52,경남 창원시 의창구 용지로169번길 26,128.681022931214,35.2320799258082
4,진해해양공원,46,경남 창원시 진해구 명동로 62,128.719347539462,35.0921817074027
5,진해내수면환경생태공원,37,경남 창원시 진해구 여명로71번길 65,128.660005160088,35.1640346085716
6,가포해안변공원,34,경남 창원시 마산합포구 가포동 산 82,128.589396857646,35.1583373818969
7,창원단감테마공원,30,경남 창원시 의창구 동읍 화양리 680,0,0
8,제황산공원,23,경남 창원시 진해구 제황산동 28-5,128.664434312791,35.1476322490856
9,사화공원,14,경남 창원시 의창구 중동 145,0,0


In [23]:
# 창원단감테마공원, 사화공원, 산호공원의 경도, 위도값이 api로 구해지지 않아서 직접 찾아 메꿈 
park_coorinate.loc[7,'longtitude'] = '128.663353338049'
park_coorinate.loc[7,'latitude'] = '35.2985613214767'

park_coorinate.loc[9,'longtitude'] = '128.627755158373'
park_coorinate.loc[9,'latitude'] = '35.2522724542047'

park_coorinate.loc[12,'longtitude'] = '128.581540595384'
park_coorinate.loc[12,'latitude'] = '35.2141323947341'

In [24]:
park_coorinate

,park,count,address,longtitude,latitude
0,북면수변생태공원,150,경남 창원시 의창구 북면 외산리 108,128.609790815343,35.3805879621255
1,창원해양공원,99,경남 창원시 진해구 명동로 62,128.719347539462,35.0921817074027
2,장미공원,69,경남 창원시 성산구 가음동 31,128.690275383692,35.2074243862455
3,용지공원,52,경남 창원시 의창구 용지로169번길 26,128.681022931214,35.2320799258082
4,진해해양공원,46,경남 창원시 진해구 명동로 62,128.719347539462,35.0921817074027
5,진해내수면환경생태공원,37,경남 창원시 진해구 여명로71번길 65,128.660005160088,35.1640346085716
6,가포해안변공원,34,경남 창원시 마산합포구 가포동 산 82,128.589396857646,35.1583373818969
7,창원단감테마공원,30,경남 창원시 의창구 동읍 화양리 680,128.663353338049,35.2985613214767
8,제황산공원,23,경남 창원시 진해구 제황산동 28-5,128.664434312791,35.1476322490856
9,사화공원,14,경남 창원시 의창구 중동 145,128.627755158373,35.2522724542047


In [25]:
park_coorinate.loc[1,'count'] = 99+46
park_coorinate

,park,count,address,longtitude,latitude
0,북면수변생태공원,150,경남 창원시 의창구 북면 외산리 108,128.609790815343,35.3805879621255
1,창원해양공원,145,경남 창원시 진해구 명동로 62,128.719347539462,35.0921817074027
2,장미공원,69,경남 창원시 성산구 가음동 31,128.690275383692,35.2074243862455
3,용지공원,52,경남 창원시 의창구 용지로169번길 26,128.681022931214,35.2320799258082
4,진해해양공원,46,경남 창원시 진해구 명동로 62,128.719347539462,35.0921817074027
5,진해내수면환경생태공원,37,경남 창원시 진해구 여명로71번길 65,128.660005160088,35.1640346085716
6,가포해안변공원,34,경남 창원시 마산합포구 가포동 산 82,128.589396857646,35.1583373818969
7,창원단감테마공원,30,경남 창원시 의창구 동읍 화양리 680,128.663353338049,35.2985613214767
8,제황산공원,23,경남 창원시 진해구 제황산동 28-5,128.664434312791,35.1476322490856
9,사화공원,14,경남 창원시 의창구 중동 145,128.627755158373,35.2522724542047


In [29]:
# 창원해양공원과 진해해양공원은 같은 곳이므로 합병 
park_coorinate.drop(4,inplace=True)

In [30]:
park_coorinate

,park,count,address,longtitude,latitude
0,북면수변생태공원,150,경남 창원시 의창구 북면 외산리 108,128.609790815343,35.3805879621255
1,창원해양공원,145,경남 창원시 진해구 명동로 62,128.719347539462,35.0921817074027
2,장미공원,69,경남 창원시 성산구 가음동 31,128.690275383692,35.2074243862455
3,용지공원,52,경남 창원시 의창구 용지로169번길 26,128.681022931214,35.2320799258082
5,진해내수면환경생태공원,37,경남 창원시 진해구 여명로71번길 65,128.660005160088,35.1640346085716
6,가포해안변공원,34,경남 창원시 마산합포구 가포동 산 82,128.589396857646,35.1583373818969
7,창원단감테마공원,30,경남 창원시 의창구 동읍 화양리 680,128.663353338049,35.2985613214767
8,제황산공원,23,경남 창원시 진해구 제황산동 28-5,128.664434312791,35.1476322490856
9,사화공원,14,경남 창원시 의창구 중동 145,128.627755158373,35.2522724542047
10,본포수변생태공원,5,경남 창원시 의창구 동읍 본포리,128.649454269739,35.3658963163472


In [31]:
park_coorinate.to_csv('C:\\Users\\user1\\Desktop\\창원_빅데이터_공모전\\Changwon_BigData_Competition\\Preprocessed_data\\Changwon_park_coordinates.csv')

In [11]:
os.getcwd()

'C:\\Users\\user1\\Desktop\\창원_빅데이터_공모전\\Changwon_BigData_Competition\\Code_analyzing'